In [19]:
import matplotlib
matplotlib.use('Agg')
import sys
import gc
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import FormatStrFormatter

# Plots

In [33]:
# Se define el nivel al cual se obtendran las graficas
lvl = 'level3'
#lvl = 'level4'

# Los niveles tienen un conjunto distinto de halos
halonums = None
if lvl == 'level3':
    halonums = [6,16,21,23,24,27]
if lvl == 'level4':
    halonums = range(1,31)


# Graficas exportadas a pdf
with PdfPages('../Plots/DM_VS_MHD_'+lvl+'.pdf') as pdf:
    for i in halonums:
        # Obtiene el nombre de la carpeta para el halo especifico
        halo = 'halo_'+str(i)

        # Consulta los semiejes de DM y MHD
        arrDM = np.loadtxt("../Plots/"+lvl+"_DM/"+halo+"/"+"abc_"+lvl+"_DM_"+halo+".txt", delimiter = ',')
        arrMHD = np.loadtxt("../Plots/"+lvl+"_MHD/"+halo+"/"+"abc_"+lvl+"_MHD_"+halo+".txt", delimiter = ',')

        # Obtiene los semiejes (planos y sin reescalar)
        aDM,bDM,cDM = arrDM[:-1].T
        aMHD,bMHD,cMHD = arrMHD[:-1].T

        # Obtiene las fracciones de los semiejes
        yvalsDM = np.array([bDM/aDM,cDM/aDM,(1-(bDM/aDM)**2)/(1-(cDM/aDM)**2)])  
        xvalsDM = (aDM*bDM*cDM)**(1./3.)
        yvalsMHD = np.array([bMHD/aMHD,cMHD/aMHD,(1-(bMHD/aMHD)**2)/(1-(cMHD/aMHD)**2)])
        xvalsMHD = (aMHD*bMHD*cMHD)**(1./3.)

        # Label de los ejes
        ylabel = ['b/a','c/a','T']

        # Define el radio virial 
        rvirDM = arrDM[-1][0]
        rvirMHD = arrMHD[-1][0]

        # La figura, cambiar tamano
        fig, axs = plt.subplots(figsize=(10,7),nrows=len(yvalsDM))
        for ax,yvalDM,yvalMHD,ylab in zip(axs,yvalsDM,yvalsMHD,ylabel):
            ax.plot(xvalsDM,yvalDM, c = 'darkblue',linestyle = '--', label = "DM")
            ax.plot(xvalsMHD,yvalMHD, c = 'darkred', label = "MHD")
            ax.plot([rvirDM,rvirDM],[0,1],c = 'darkblue',linestyle = '--')
            ax.plot([rvirMHD,rvirMHD],[0,1], c = 'darkred')
            ax.set_xscale('log')
            # Plotting ratios
            ax.set_ylim(0,1)
            # Valid for all Milkyway-like galaxies
            ax.set_xlim(0.1,rvirDM+30)
            ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
            ax.set_ylabel(ylab)
        axs[-1].set_xlabel("log(R(kpc/h))")
        pdf.savefig(fig)
        plt.close()